**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов (NLTK model)

In [1]:
import re
import gzip
from random import sample
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [2]:
file_name = '../data/dostoevsky-besy-p2.txt.gz'

with gzip.open(file_name,'rt') as f:  text = f.read()[105:] # выкидываем заголовок

print('символов:%i\n'%(len(text)))
print(text[:364].strip())

text = [ 
    word_tokenize (s) # разбиваем предложения на слова
    for s in sent_tokenize(text) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(text)))

sample(text,2)

символов:465490

Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.
предложений: 5556



[['-', 'Нет', '.'],
 ['Шатов', 'брезгливо', 'и', 'раздражительно', 'ухмылялся', '.']]

In [3]:
from nltk.lm.preprocessing import padded_everygram_pipeline

# генерируем учебный датасет
train, vocab = padded_everygram_pipeline(2, text)


# собираем модель

from nltk.lm import MLE as Model # Maximum Likelihood Estimator
# from nltk.lm import Laplace as Model

model = Model(2) 
model.fit(train, vocab)
print(len(model.vocab))

16634


In [4]:
# генерируем продолжения
for sentence in sample(text,10):
    if len(sentence)<5: continue
    print( 
        ' '.join(sentence[:-2])
        + '...' 
        + str( model.generate(3, text_seed=sentence[:-2]) )
        + '\n'
    )

Вас бы я конечно не потащил туда , зная ваш теперешний образ мыслей ... то-есть в том смысле , чтобы вас там не мучить , а не из того , что мы думаем , что вы...['и', 'с', 'Кирилловым']

Швейцар меня знал ; я объявил , что мы оба к Юлии...['Михайловны', 'на', 'след']

Петр Степанович , конечно , знал , что рискует , пускаясь в такие выверты , но уж когда он сам бывал возбужден , то лучше желал рисковать хоть на все , чем оставлять себя в...['котором', 'я', 'не']

- Ну да , ну...['там', 'есть', 'тут']

Недели с три назад заболел там и умер один рабочий азиятскою холерой ; потом заболело еще несколько...['камней', 'и', 'не']

Начиналась она грозными звуками Марсельезы : '' Qu'un sang impur abreuve nos sillons...['!', '</s>', 'с']

Минуту спустя он проговорил , смотря на меня в отчаянии : - Я...['сама', 'Юлия', 'Михайловна']

Этот пристав - восторженно административная личность , Василий Иванович Флибустьеров , был еще недавним гостем в нашем городе , но уже отличился и прогремел своею не

In [5]:
from nltk.util import bigrams

text_bigrams = [ ngram for s in text for ngram in bigrams(s) ]

In [6]:
model.entropy( text_bigrams  )

4.56330074985801

In [7]:
model.perplexity( text_bigrams )

23.642337077815522